# LlamaCpp

https://llama-cpp-python.readthedocs.io/en/latest/api-reference/#llama_cpp.Llama.generate

In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
# from langchain_community.llms import LlamaCpp
from langchain.llms import LlamaCpp

In [2]:
# models/model-q4_K.gguf - https://huggingface.co/IlyaGusev/saiga_mistral_7b_gguf/tree/main
llm = LlamaCpp(
    model_path="models/model-q4_K.gguf ",
    # callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from models/model-q4_K.gguf  (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32
ll

## Обычный промпт

In [3]:
template = """Вопрос: {question}

Ответ:"""

prompt = PromptTemplate.from_template(template)
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
prompt

PromptTemplate(input_variables=['question'], template='Вопрос: {question}\n\nОтвет:')

In [4]:
# или так
prompt_template = PromptTemplate(
        input_variables=['question'],
        template=template,
    )
prompt_template

PromptTemplate(input_variables=['question'], template='Вопрос: {question}\n\nОтвет:')

In [5]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain

LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Вопрос: {question}\n\nОтвет:'), llm=LlamaCpp(client=<llama_cpp.llama.Llama object at 0x00000204421D5E20>, model_path='models/model-q4_K.gguf '))

In [6]:
%%time
question = "Какой спутник у планеты Земля?"
result = llm_chain.run(question)


llama_print_timings:        load time =   11108.63 ms
llama_print_timings:      sample time =       1.95 ms /     4 runs   (    0.49 ms per token,  2049.18 tokens per second)
llama_print_timings: prompt eval time =   14773.02 ms /    24 tokens (  615.54 ms per token,     1.62 tokens per second)
llama_print_timings:        eval time =    1677.32 ms /     4 runs   (  419.33 ms per token,     2.38 tokens per second)
llama_print_timings:       total time =   16510.69 ms /    28 tokens


CPU times: total: 39.1 s
Wall time: 16.5 s


In [7]:
print(result)

 Луна.


## промпт под задачу NER

In [8]:
# models/llama-2-7b-chat.Q3_K_M.gguf - https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/tree/main
llm = LlamaCpp(
    model_path="models/llama-2-7b-chat.Q3_K_M.gguf",
    # callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from models/llama-2-7b-chat.Q3_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32         

In [10]:
llm

LlamaCpp(client=<llama_cpp.llama.Llama object at 0x00000204421D5280>, model_path='models/llama-2-7b-chat.Q3_K_M.gguf')

In [11]:
template = """[INST] <<SYS>>
Ты специалист по задаче извлечения сущностей - named entity recognition. 
Ты извлекаешь из предложения {sentence} все сущности, которые есть в списке {entities}.
Сущностей, которых нет в этом списке, ты не извлекаешь.
Ответы ты даешь на русском языке.
<</SYS>>

Извлеки из предложения {sentence} сущности {entities}[/INST]
"""
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['entities', 'sentence'], template='[INST] <<SYS>>\nТы специалист по задаче извлечения сущностей - named entity recognition. \nТы извлекаешь из предложения {sentence} все сущности, которые есть в списке {entities}.\nСущностей, которых нет в этом списке, ты не извлекаешь.\nОтветы ты даешь на русском языке.\n<</SYS>>\n\nИзвлеки из предложения {sentence} сущности {entities}[/INST]\n')

In [12]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
llm_chain

LLMChain(prompt=PromptTemplate(input_variables=['entities', 'sentence'], template='[INST] <<SYS>>\nТы специалист по задаче извлечения сущностей - named entity recognition. \nТы извлекаешь из предложения {sentence} все сущности, которые есть в списке {entities}.\nСущностей, которых нет в этом списке, ты не извлекаешь.\nОтветы ты даешь на русском языке.\n<</SYS>>\n\nИзвлеки из предложения {sentence} сущности {entities}[/INST]\n'), llm=LlamaCpp(client=<llama_cpp.llama.Llama object at 0x00000204421D5280>, model_path='models/llama-2-7b-chat.Q3_K_M.gguf'))

In [11]:
sentence = "Иванов Сергей Владимирович, г. Москва, ул. Люсиновская 10, кв. 4"
entities = ['surname', 'street']
input_variables_dict = {'sentence': sentence, 'entities': entities}
# input_variables_dict = {'sentence': sentence}
result = llm_chain.run(input_variables_dict)
print(result)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    9095.27 ms
llama_print_timings:      sample time =     313.33 ms /    74 runs   (    4.23 ms per token,   236.18 tokens per second)
llama_print_timings: prompt eval time =   43431.88 ms /   187 tokens (  232.26 ms per token,     4.31 tokens per second)
llama_print_timings:        eval time =   25297.19 ms /    73 runs   (  346.54 ms per token,     2.89 tokens per second)
llama_print_timings:       total time =   71115.17 ms /   260 tokens


Из предложения "Иванов Сергей Владимирович, г. Москва, ул. Люсиновская 10, кв. 4" мы можем извлечь следующие сущности:
* 'surname' - 'Иванов'
* 'street' - 'Люсиновская'
